## Local Inference on GPU
Model page: https://huggingface.co/MedVita/medgemma-27b-lora-swift

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/MedVita/medgemma-27b-lora-swift)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("google/medgemma-27b-it")
model = PeftModel.from_pretrained(base_model, "MedVita/medgemma-27b-lora-swift")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/medgemma-27b-it.
401 Client Error. (Request ID: Root=1-692efff8-25d6a8312a43ff2975eea23b;25f044ca-8178-4482-b9ac-d0e2f29d9305)

Cannot access gated repo for url https://huggingface.co/google/medgemma-27b-it/resolve/main/config.json.
Access to model google/medgemma-27b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_name = "sairajkadam/MedViT-small"

from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=4)


In [2]:
!nvidia-smi

Tue Dec  2 15:04:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!git clone https://github.com/Omid-Nejati/MedViT.git

fatal: destination path 'MedViT' already exists and is not an empty directory.


In [4]:
!cd /content/MedViT

In [5]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary

In [6]:
!pip install timm
!pip install einops

In [7]:
import sys
sys.path.append('/content/MedViT')

import importlib.util

medvit_path = '/content/MedViT/MedViT.py'
spec = importlib.util.spec_from_file_location("medvit_model_module", medvit_path)
medvit_model_module = importlib.util.module_from_spec(spec)
sys.modules["medvit_model_module"] = medvit_model_module
spec.loader.exec_module(medvit_model_module)

tiny = medvit_model_module.MedViT_small

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [8]:
!ls -F /content/MedViT

Colab_MedViT.ipynb  images/		MedViT.py     requirements.txt
CustomDataset/	    Instructions.ipynb	__pycache__/  utils.py
CustomDataset.md    LICENSE		README.md


In [9]:
!cat /content/MedViT/MedViT.py

"""
Author: Omid Nejati
Email: omid_nejaty@alumni.iust.ac.ir

MedViT: A Robust Vision Transformer for Generalized Medical Image Classification.
"""
from functools import partial
import math
import torch
import torch.utils.checkpoint as checkpoint
from einops import rearrange
from timm.models.layers import DropPath, trunc_normal_
from timm.models.registry import register_model
from torch import nn
from utils import merge_pre_bn

NORM_EPS = 1e-5


class ConvBNReLU(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            stride,
            groups=1):
        super(ConvBNReLU, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride,
                              padding=1, groups=groups, bias=False)
        self.norm = nn.BatchNorm2d(out_channels, eps=NORM_EPS)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
  

In [31]:
model = tiny(num_classes=n_classes)

initialize_weights...


In [32]:
model.proj_head[0]

Linear(in_features=1024, out_features=4, bias=True)

In [27]:
# The model's final layer is already configured with the correct number of classes during instantiation, so this line is no longer needed.

In [33]:
if torch.cuda.is_available():
    model = model.cuda()
else:
    print("CUDA is not available. Using CPU.")

In [14]:
dataset="/content/drive/MyDrive/CKDupdated.zip"
dataset

'/content/drive/MyDrive/CKDupdated.zip'

In [15]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

In [16]:
NUM_EPOCHS = 10
BATCH_SIZE = 10
lr = 0.005

# The 'INFO' dictionary and 'data_flag' variable were not defined.
# Based on the model's last layer configuration, n_classes should be 2.
# If 'task' and 'n_channels' are needed, they should be defined explicitly.
# info = INFO[data_flag]
task = 'multi-class' # Defined as multi-class based on the dataset structure
# n_channels = info['n_channels']
n_classes = 4 # Inferred from the model's final linear layer output (out_features=2), but corrected to 4 based on the dataset classes

In [17]:
from torchvision.transforms.transforms import Resize
# preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)

# pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

NameError: name 'DataClass' is not defined

In [18]:
import zipfile
import os

zip_path = dataset # Use the already defined 'dataset' variable
extract_path = '/content/CKDupdated'

# Create the directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Dataset extracted to: {extract_path}')

Dataset extracted to: /content/CKDupdated


In [19]:
from torchvision.transforms.transforms import Resize
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

# preprocessing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize to a fixed square size
    transforms.Lambda(lambda image: image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]) # Normalize for 3 channels
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize to a fixed square size
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]) # Normalize for 3 channels
])

# Define the path to the unzipped dataset where class folders are directly present
data_dir = '/content/CKDupdated/CKDupdated'

# Load the entire dataset using ImageFolder
full_dataset = ImageFolder(root=data_dir, transform=train_transform)

# Define the split ratio for training and testing
train_size = int(0.8 * len(full_dataset)) # 80% for training
test_size = len(full_dataset) - train_size # Remaining for testing

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# For the test dataset, we might want to apply a different transform if AugMix is only for training
# However, since random_split creates subsets, we need to handle transforms differently.
# A common approach is to create two full datasets with different transforms and then split.
# For simplicity and direct fix, we'll keep the same transform for now unless specified otherwise.

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
# print(f"Classes: {full_dataset.classes}") # Classes can be accessed from full_dataset

Number of training samples: 1600
Number of test samples: 400


In [20]:
import os
# Check the contents of the extracted directory to confirm the correct path
print(os.listdir('/content/CKDupdated/CKDupdated'))

['stone_up', 'Normal_up', 'cyst_up', 'tumor_up']


In [29]:
# define loss function and optimizer
# task is defined as 'multi-class'
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# train

for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]'% (epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

Epoch [1/10]


100%|██████████| 160/160 [00:54<00:00,  2.94it/s]


Epoch [2/10]


100%|██████████| 160/160 [00:53<00:00,  2.98it/s]


Epoch [3/10]


...

100%|██████████| 160/160 [00:53<00:00,  2.97it/s]


Epoch [9/10]


100%|██████████| 160/160 [00:53<00:00,  2.96it/s]


Epoch [10/10]


100%|██████████| 160/160 [00:54<00:00,  2.96it/s]


In [30]:
from sklearn.metrics import roc_auc_score

# evaluation

# Define a simple Evaluator class
class Evaluator:
    def __init__(self, data_flag, split):
        self.data_flag = data_flag
        self.split = split

    def evaluate(self, y_score, y_true):
        # For multi-class, y_score will be probabilities, y_true will be class indices
        # Calculate accuracy
        y_pred = np.argmax(y_score, axis=1)
        correct_predictions = np.sum(y_pred == y_true)
        accuracy = correct_predictions / len(y_true)

        # Calculate multi-class AUC using one-vs-rest strategy with macro averaging
        # roc_auc_score requires y_true to be integers (class labels) and y_score to be probabilities.
        # The 'multi_class' parameter is set to 'ovr' (one-vs-rest) and 'average' to 'macro').
        # Explicitly pass 'labels' to handle cases where not all classes are present in y_true.
        try:
            auc = roc_auc_score(y_true, y_score, multi_class='ovr', average='macro', labels=np.arange(n_classes))
        except ValueError as e:
            print(f"Warning: Could not compute AUC. {e}")
            auc = 0.0 # Fallback in case AUC cannot be computed (e.g., single class present in batch)

        return auc, accuracy

# Define data_flag - it was not defined previously
data_flag = "CKD_dataset" # Example placeholder

def test(split):
    model.eval()
    all_y_true = []
    all_y_score = []

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            if torch.cuda.is_available():
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                # For multi-class, targets should be long (class indices)
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)

            all_y_true.append(targets.cpu().numpy())
            all_y_score.append(outputs.cpu().numpy())

        y_true_np = np.concatenate(all_y_true, axis=0)
        y_score_np = np.concatenate(all_y_score, axis=0)

        evaluator = Evaluator(data_flag, split)
        auc, accuracy = evaluator.evaluate(y_score_np, y_true_np) # Pass y_true_np to evaluator

        print('%s  auc: %.3f  acc:%.3f' % (split, auc, accuracy))


print('==> Evaluating ...')
test('train')
test('test')

==> Evaluating ...
train  auc: 1.000  acc:0.994
test  auc: 0.998  acc:0.993
